<a href="https://colab.research.google.com/github/LawTAGS/Deep-Learning-Classwork/blob/main/Assignments/DL2025_Tugas_Data_10121091_Bryan_Guok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

\begin{align}
\text{Nama}  &: \text{Bryan Guok} \\
\text{NIM}   &: \text{10121091}
\end{align}

## **Tugas 4:  Data Prepocessing**
Anda diberikan dataset mentah yang berisi informasi karyawan dari 5 jenis perusahaan berbeda (misalnya: Teknologi, Kesehatan, Media, dsb.). \\
Dataset tersebut memiliki 500 baris data mentah dengan beberapa masalah umum, seperti: \\
* Nilai kosong (missing values)
* Data non-numerik di kolom numerik
* Data duplikat
* Kolom kategorikal yang perlu diencoding
* Fitur tanggal yang belum diolah \\

Tugas Anda adalah melakukan preprocessing menyeluruh agar dataset siap digunakan
untuk proses modeling (modelling tidak perlu dilakukan).

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler

## Load Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

file_id1 = "11znOheXbT2O6Kch352p8ozsrCmvJJf6T"
file_id2 = "1MWHTrTbaT1VlMmvGot547mZwEMapFyWc"
file_id3 = "1s5Z_QfAOF-IvwjIRyukPoKGgtM8BfKGF"

file_url1 = f"https://drive.google.com/uc?id={file_id1}"
file_url2 = f"https://drive.google.com/uc?id={file_id2}"
file_url3 = f"https://drive.google.com/uc?id={file_id3}"

df1 = pd.read_csv(file_url1)
df2 = pd.read_csv(file_url2)
df3 = pd.read_csv(file_url3)

df1.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Name,Age,Salary,Department,Joined_Date,Performance
0,Employee_v1_001,NaN,sixty-five hundred thousand,HR,2018-07-13,Poor
1,Employee_v1_002,39.0,5278735,Marketing,2016-09-08,Good
2,Employee_v1_003,30.0,NaN,Finance,2017-11-10,Good
3,Employee_v1_004,30.0,6931036,Marketing,2022-11-04,Good
4,Employee_v1_005,NaN,NaN,Marketing,2019-09-23,Poor


In [ ]:
df2.head()

,Name,Age,Salary,Department,Joined_Date,Performance
0,Employee_v2_001,33.0,forty thousand,QA,2016-05-06,Poor
1,Employee_v2_002,38.0,NaN,Engineering,2017-06-28,Good
2,Employee_v2_003,24.0,NaN,Product,2019-08-30,Poor
3,Employee_v2_004,43.0,forty thousand,Support,2021-06-19,Good
4,Employee_v2_005,34.0,forty thousand,QA,2019-08-22,Poor


In [ ]:
df3.head()

,Name,Age,Salary,Department,Joined_Date,Performance
0,Employee_v3_001,NaN,NaN,Sales,2021-09-27,Poor
1,Employee_v3_002,NaN,NaN,Logistics,2023-03-13,Good
2,Employee_v3_003,37.0,NaN,Procurement,2022-07-10,Poor
3,Employee_v3_004,40.0,NaN,Sales,2021-07-12,Good
4,Employee_v3_005,44.0,forty thousand,Logistics,2016-07-10,Good


## **1. Data Cleansing**
Akan dibuat dalam bentuk fungsi yang *reusable* agar dapat diapply ke semua *dataframe* dengan mudah

### Fungsi untuk mengubah kata-kata menjadi angka

In [ ]:
# Terlebih dahulu dibuat library yang mem-pair antara angka dan kata-kata dalam
# bahasa Inggris
number_words = {
    "zero": 0, "one": 1, "two": 2, "three": 3, "four": 4,
    "five": 5, "six": 6, "seven": 7, "eight": 8, "nine": 9,
    "ten": 10, "eleven": 11, "twelve": 12, "thirteen": 13,
    "fourteen": 14, "fifteen": 15, "sixteen": 16, "seventeen": 17,
    "eighteen": 18, "nineteen": 19, "twenty": 20, "thirty": 30,
    "forty": 40, "fifty": 50, "sixty": 60, "seventy": 70,
    "eighty": 80, "ninety": 90, "hundred": 100, "thousand": 1000,
    "million": 1_000_000
}

# Kemudian nilainya akan dikonversi menjadi angka dengan satuan berdasarkan posisi katanya
def parse_salary_words(text):
    words = str(text).lower().replace('-', ' ').split()
    total = 0
    current = 0

    for word in words:
        if word in number_words:
            val = number_words[word]
            if val == 100:
                if current == 0: current = 1
                current *= 100
            elif val >= 1000:
                if current == 0: current = 1
                current *= val
                total += current
                current = 0
            else:
                current += val
        else:
            # Skip unknown words
            continue

# Nilainya kemudian dijumlahkan kembali agar utuh
    total += current
    return total if total > 0 else None

### Fungsi untuk Membersihkan Keseluruhan Data

In [ ]:
def preprocess_employee_data(df):
    df = df.copy()  # Menghindari mengubah dataframe yang asli

    # 1. Mengubah Nilai Kosong
    # Nilai umur yang kosong akan diisi median umur dari data
    df["Age"].fillna(df["Age"].median(), inplace=True)

    # Membuat fungsi menggunakan registry editor untuk memfilter format salary yang tidak sesuai
    # Idenya registry editor bisa memfilter section yang ada angkanya aja
    # kemudian diubah jadi salary, namun untuk format yang persis hanya kata-kata
    # tidak ada numeriknya sama sekali akan dicoba untuk diubah menggunakan
    # fungsi sebelumnya
    def safe_salary_parser(val):
        try:
            return float(val)
        except:
            matches = re.findall(r'\d+', str(val))
            if matches:
                return float("".join(matches))
            else:
                # Gunakan parser manual dari kata
                return parse_salary_words(val)

    # Meng-apply fungsi filtrasi yang sudah dibuat ke dataframe bagian Salary
    # Apabila ada yang kosong maka nilai akan diisi dengan mean dari Salary
    df["Salary_Numeric"] = df["Salary"].apply(safe_salary_parser)
    df["Salary_Numeric"].fillna(df["Salary_Numeric"].mean(), inplace=True)
    df.drop(columns=["Salary"], inplace=True)
    df.rename(columns={"Salary_Numeric": "Salary"}, inplace=True)

    # 2. Feature Engineering
    # Karena patokan tanggal saat ini dianggap dari 01 Januari 2025, maka selisih
    # bulan dan tanggal dapat diabaikan dalam perhitungan tahun pelayanan
    # akibatnya dapat langsung dihitung 2025 - tahun join
    df["Joined_Date"] = pd.to_datetime(df["Joined_Date"], errors='coerce')
    df["Years_of_Service"] = 2025 - df["Joined_Date"].dt.year

    # 3. Encoding
    # Menerapkan Label Encoding pada kolom department agar kategori department
    # diubah menjadi numerik
    if "Department" in df.columns:
        df["Department_Encoded"] = LabelEncoder().fit_transform(df["Department"])

    # Menerapkan One-Hot Encoding menggunakan pd.get_dummies() untuk mengubah
    # setiap kategori menjadi kolom binary dengan prefix dari kolomnya adalah "Perf-".
    # Contoh: Perf-High, Perf-Medium, Perf-Low
    # Kemudian digabungkan ke dalam dataframe
    if "Performance" in df.columns:
        performance_dummies = pd.get_dummies(df["Performance"], prefix="Perf")
        df = pd.concat([df, performance_dummies], axis=1)

    # 4. Scaling
    # Menerapkan Min-Max Scaler pada tiap kolom yang berguna
    scaler = MinMaxScaler()
    for col in ["Age", "Salary", "Years_of_Service"]:
        if col in df.columns:
            df[f"{col}_scaled"] = scaler.fit_transform(df[[col]])

    return df

In [ ]:
# Apply preprocessing to each company dataset
cleaned_df1 = preprocess_employee_data(df1)
cleaned_df2 = preprocess_employee_data(df2)
cleaned_df3 = preprocess_employee_data(df3)

# Optional: Save to CSV
# cleaned_df1.to_csv("cleaned_company1.csv", index=False)
# cleaned_df2.to_csv("cleaned_company2.csv", index=False)
# cleaned_df3.to_csv("cleaned_company3.csv", index=False)


<ipython-input-27-a098017dd995>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Age"].fillna(df["Age"].median(), inplace=True)
<ipython-input-27-a098017dd995>:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u

,Name,Age,Department,Joined_Date,Performance,Salary,Years_of_Service,Department_Encoded,Perf_Good,Perf_Poor,Age_scaled,Salary_scaled,Years_of_Service_scaled
0,Employee_v1_001,33.0,HR,2018-07-13,Poor,6.500000e+06,7,1,False,True,0.428571,0.721302,0.666667
1,Employee_v1_002,39.0,Marketing,2016-09-08,Good,5.278735e+06,9,3,True,False,0.714286,0.584940,0.888889
2,Employee_v1_003,30.0,Finance,2017-11-10,Good,4.512302e+06,8,0,True,False,0.285714,0.499362,0.777778
3,Employee_v1_004,30.0,Marketing,2022-11-04,Good,6.931036e+06,3,3,True,False,0.285714,0.769430,0.222222
4,Employee_v1_005,33.0,Marketing,2019-09-23,Poor,4.512302e+06,6,3,False,True,0.428571,0.499362,0.555556


In [ ]:
cleaned_df1.head()

,Name,Age,Department,Joined_Date,Performance,Salary,Years_of_Service,Department_Encoded,Perf_Good,Perf_Poor,Age_scaled,Salary_scaled,Years_of_Service_scaled
0,Employee_v1_001,33.0,HR,2018-07-13,Poor,6.500000e+06,7,1,False,True,0.428571,0.721302,0.666667
1,Employee_v1_002,39.0,Marketing,2016-09-08,Good,5.278735e+06,9,3,True,False,0.714286,0.584940,0.888889
2,Employee_v1_003,30.0,Finance,2017-11-10,Good,4.512302e+06,8,0,True,False,0.285714,0.499362,0.777778
3,Employee_v1_004,30.0,Marketing,2022-11-04,Good,6.931036e+06,3,3,True,False,0.285714,0.769430,0.222222
4,Employee_v1_005,33.0,Marketing,2019-09-23,Poor,4.512302e+06,6,3,False,True,0.428571,0.499362,0.555556


In [ ]:
cleaned_df2.head()

,Name,Age,Department,Joined_Date,Performance,Salary,Years_of_Service,Department_Encoded,Perf_Good,Perf_Poor,Age_scaled,Salary_scaled,Years_of_Service_scaled
0,Employee_v2_001,33.0,QA,2016-05-06,Poor,4.000000e+04,9,2,False,True,0.428571,0.000000,0.888889
1,Employee_v2_002,38.0,Engineering,2017-06-28,Good,4.348553e+06,8,0,True,False,0.666667,0.482908,0.777778
2,Employee_v2_003,24.0,Product,2019-08-30,Poor,4.348553e+06,6,1,False,True,0.000000,0.482908,0.555556
3,Employee_v2_004,43.0,Support,2021-06-19,Good,4.000000e+04,4,3,True,False,0.904762,0.000000,0.333333
4,Employee_v2_005,34.0,QA,2019-08-22,Poor,4.000000e+04,6,2,False,True,0.476190,0.000000,0.555556


In [ ]:
cleaned_df3.head()

,Name,Age,Department,Joined_Date,Performance,Salary,Years_of_Service,Department_Encoded,Perf_Good,Perf_Poor,Age_scaled,Salary_scaled,Years_of_Service_scaled
0,Employee_v3_001,34.0,Sales,2021-09-27,Poor,4.341418e+06,4,3,False,True,0.476190,0.482716,0.333333
1,Employee_v3_002,34.0,Logistics,2023-03-13,Good,4.341418e+06,2,1,True,False,0.476190,0.482716,0.111111
2,Employee_v3_003,37.0,Procurement,2022-07-10,Poor,4.341418e+06,3,2,False,True,0.619048,0.482716,0.222222
3,Employee_v3_004,40.0,Sales,2021-07-12,Good,4.341418e+06,4,3,True,False,0.761905,0.482716,0.333333
4,Employee_v3_005,44.0,Logistics,2016-07-10,Good,4.000000e+04,9,1,True,False,0.952381,0.000000,0.888889
